In [3]:
import os
import pandas
import regex as re
import time
from tqdm import tnrange, tqdm_notebook
import tqdm
import numpy
import torch
from scipy.sparse import coo_matrix

In [4]:
path = os.getcwd() + '/raw_name'
path

'/home/guru/Desktop/GNN-Call-Graphs/raw_name'

In [5]:
print(os.listdir())

['Readme.md', 'construction.ipynb', 'Dataset', '.ipynb_checkpoints', 'raw_name', 'test.py', '.git', 'GCN - Approach 1.py', '.idea', 'train.py', 'tensorflow-env.yaml', 'results - 1 AUC curve.txt', 'networkx visualisation.ipynb', 'torch-env.yaml']


In [6]:
# file = open('new.txt', 'w')
# data = 'a \n b \n c'
# file.writelines(data)
# file.close()
# file = open('new.txt', 'r')
# print(file.read())

In [7]:
path

'/home/guru/Desktop/GNN-Call-Graphs/raw_name'

In [8]:
os.chdir(path)
df = []
def read_text_file(file_path):
    file = open(file_path, 'r')
    for line in file:
        df.append(line)
  
  
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
  
        # call read text file function
        read_text_file(file_path)

In [9]:
len(df)

12074

In [10]:
df[0:3]

['org.apache.maven.artifact.factory.DefaultArtifactFactory.createArtifact org.apache.maven.artifact.versioning.VersionRange.createFromVersion\n',
 'org.apache.maven.artifact.factory.DefaultArtifactFactory.createArtifact org.apache.maven.artifact.handler.manager.ArtifactHandlerManager.getArtifactHandler\n',
 'org.apache.maven.artifact.repository.DefaultRepositoryRequest.getRepositoryRequest org.apache.maven.artifact.repository.RepositoryRequest.setLocalRepository\n']

In [11]:
lst = []
lst.append("a.b c.dsf.dd")

In [12]:
lst[0]

'a.b c.dsf.dd'

In [13]:
dn = '(\w\n ,'

In [14]:
df[0]

'org.apache.maven.artifact.factory.DefaultArtifactFactory.createArtifact org.apache.maven.artifact.versioning.VersionRange.createFromVersion\n'

In [15]:
# spilliting
x = df[0].split()
x

['org.apache.maven.artifact.factory.DefaultArtifactFactory.createArtifact',
 'org.apache.maven.artifact.versioning.VersionRange.createFromVersion']

In [16]:
pattern1 = r'\w*\s*\w+(?=\.)'
pattern2 = '(?<=\.)\w*\s*\w+$'
ptrn = [pattern1, pattern2]
ptn = re.compile(r'(?<=\.)\w*\s*\w+$')

In [17]:
matches = []
for pat in ptrn:
    matches += re.findall(pat, x[0])
    
matches

['org',
 'apache',
 'maven',
 'artifact',
 'factory',
 'DefaultArtifactFactory',
 'createArtifact']

In [18]:
# re.findall('(%s|%s)' % (pattern1,pattern2), x[0] )
re.compile("(%s|%s)" % (pattern1, pattern2)).findall(x[0])


['org',
 'apache',
 'maven',
 'artifact',
 'factory',
 'DefaultArtifactFactory',
 'createArtifact']

In [19]:
# split using regex
re.split(r"\.", x[0])

['org',
 'apache',
 'maven',
 'artifact',
 'factory',
 'DefaultArtifactFactory',
 'createArtifact']

In [20]:
count = 0
for i in df:
    count = count + 1
count*2

24148

In [21]:
lst = []
count = 0
for combined in df:
    comb = combined.split()
    
    for sentence in comb:
        lst += re.compile('(%s|%s)' % (pattern1, pattern2)).findall(sentence)
        count = count + 1
        

In [22]:
len(lst), count

(154390, 24148)

In [23]:
print(lst[0:15])

['org', 'apache', 'maven', 'artifact', 'factory', 'DefaultArtifactFactory', 'createArtifact', 'org', 'apache', 'maven', 'artifact', 'versioning', 'VersionRange', 'createFromVersion', 'org']


In [24]:
lst = list(set(lst))

In [25]:
# lst

In [26]:
# we have to make a list of list 
splitted_df = []

for i in df:
    comb = i.split()
    for sp in comb:
        splitted_df.append(sp)
    

In [27]:
len(splitted_df)

24148

In [28]:
matrix = []
for call in splitted_df:
    call_break = re.split(r'\.', call)
    print(call_break)
    ans = []
    for word in lst:
        if word in call_break:
            ans.append(1)
        else:
            ans.append(0)
    
    matrix.append(ans)
    break

['org', 'apache', 'maven', 'artifact', 'factory', 'DefaultArtifactFactory', 'createArtifact']


In [29]:
count = 0
for i in matrix[0]:
    if(i == 1):
        count = count+1

# check if size is same of not
# size of first call break = 7
print(count == 7)

True


In [30]:
matrix = []
with tqdm.tqdm(total=len(splitted_df)) as t:
    for call in splitted_df:
        call_break = re.split(r'\.', call)
        ans = []
        for word in lst:
            if word in call_break:
                ans.append(1)
            else:
                ans.append(0)

        matrix.append(ans)
        t.update(1)

print("done!!!!!!")

100%|██████████| 24148/24148 [00:08<00:00, 2995.56it/s]

done!!!!!!


In [31]:
len(matrix)

24148

In [32]:
len(matrix[0]), len(matrix[24140])

(856, 856)

In [33]:
# convert list to np.array
# cool code
# length = max(map(len, x))
# y=numpy.array([xi+[None]*(length-len(xi)) for xi in x])

matrix_array = numpy.array([numpy.array(xi) for xi in matrix])
type(matrix_array)

numpy.ndarray

In [34]:
matrix_array, type(matrix_array[0])

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 numpy.ndarray)

In [35]:
# convert this node feature matrix to tensor
# node_feature_tensor = torch.tensor(matrix_array, dtype = torch.float64)

In [36]:
# This is for COO FORMAT

In [37]:
# node_feature_coo = coo_matrix(matrix_array)  

In [38]:
# 1/0 ratio
count = 0
cnt = 0
for i in matrix_array: 
    for idx in i: 
        if(idx == 1):
            count = count + 1
        if(idx == 0):
            cnt = cnt + 1

count/cnt

0.007525236765424249

In [39]:
# THIS IS FINAL MATRIX IT'S NP ARRAY OF NP ARRAY
# each row for each corrosponding row in database
matrix_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
from matplotlib import transforms
import networkx as nx
import numpy as np
import pandas as pd
import os
import sys
from glob import glob
import torch
from torch_geometric.nn import GCNConv
import os.path as osp
from torch_geometric.data import Dataset, Data
from torch_geometric.utils import train_test_split_edges
from torch_geometric.loader import DataLoader
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score
import torch_geometric.transforms as T

In [44]:
os.chdir("../")

In [45]:
os.getcwd()

'/home/guru/Desktop/GNN-Call-Graphs'

In [46]:
device = "cpu"

root = "Dataset/callNetwork"


class CallNetworks(Dataset):
    def __init__(self, root, tranform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, tranform, pre_transform, pre_filter)
    @property
    def raw_file_names(self):
        files = os.listdir(self.root+"/raw")
        return files
    @property
    def processed_file_names(self):
        processed_files = [f"data_{i}.pt" for i in range(len(self.raw_file_names))]
        return processed_files
    def process(self):
        index = 0
        for raw_path in self.raw_paths:
            print(raw_path)
            temp = list(np.unique(pd.read_csv(raw_path, delimiter=" ").values.reshape((1,-1))))
            G = nx.read_edgelist(raw_path, delimiter=" ",create_using = nx.Graph)
            # print(G.nodes)
            # print(temp)
            tmp_nodes = list(G.nodes)
            for i in temp:
                if i not in tmp_nodes:
                    G.add_node((i))
            x = torch.zeros((len(G.nodes), 5))
            edge_index = torch.from_numpy(pd.read_csv(raw_path, delimiter=" ").values).reshape((2,-1))
            for i, key in enumerate(dict(G.nodes).keys()):
                for j in range(matrix.shape[1]):
                    x[i,j] = matrix[i,j]
            data = Data(x=x, edge_index=edge_index)
            torch.save(data, osp.join(self.processed_dir,f"data_{index}.pt"))
            index += 1
    def len(self):
        return len(self.processed_file_names)
    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f"data_{idx}.pt"))
        return data

transform = T.RandomLinkSplit(num_val=0.05, num_test=0.1,is_undirected=True, add_negative_train_samples=True)
dataset = CallNetworks(root=root)
train_data, val_data, test_data = transform(dataset[0])


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)#.relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)


Epoch: 001, Loss: 13.2362, Val: 0.8302, Test: 0.8667
Epoch: 002, Loss: 3.8172, Val: 0.8147, Test: 0.8775
Epoch: 003, Loss: 3.3520, Val: 0.8403, Test: 0.8803
Epoch: 004, Loss: 4.2063, Val: 0.8679, Test: 0.8799
Epoch: 005, Loss: 3.8219, Val: 0.8651, Test: 0.8849
Epoch: 006, Loss: 2.7308, Val: 0.8325, Test: 0.8863
Epoch: 007, Loss: 1.6367, Val: 0.7985, Test: 0.8686
Epoch: 008, Loss: 1.3106, Val: 0.7960, Test: 0.8736
Epoch: 009, Loss: 1.3712, Val: 0.7933, Test: 0.8744
Epoch: 010, Loss: 1.5049, Val: 0.7898, Test: 0.8712
Epoch: 011, Loss: 1.3687, Val: 0.7880, Test: 0.8582
Epoch: 012, Loss: 1.3473, Val: 0.7848, Test: 0.8439
Epoch: 013, Loss: 1.1388, Val: 0.7684, Test: 0.8437
Epoch: 014, Loss: 1.0572, Val: 0.7700, Test: 0.8376
Epoch: 015, Loss: 0.9695, Val: 0.7700, Test: 0.8451
Epoch: 016, Loss: 0.8589, Val: 0.7850, Test: 0.8456
Epoch: 017, Loss: 0.8613, Val: 0.7860, Test: 0.8581
Epoch: 018, Loss: 0.8395, Val: 0.7827, Test: 0.8591
Epoch: 019, Loss: 0.7916, Val: 0.7994, Test: 0.8602
Epoch: 020,